In [1]:
import pickle
import os
import numpy as np

In [2]:
import torch
import torch.nn as nn

In [3]:
from utils import EnergyDemandDataset, AverageMeter

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

In [5]:
val_loader = DataLoader(EnergyDemandDataset('val', 24*7), batch_size = 16, shuffle = False)
# seq, target = next(iter(val_loader))

In [6]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers=1, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        h, _ = self.lstm(x)
        x = self.linear(h[:,-1,:])
        return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model = SimpleLSTM(1,256).to(device)
# lstm(seq.unsqueeze(2)).shape

In [38]:
target.squeeze(1).shape

torch.Size([32, 24])

In [28]:
output = model(seq[:,-168:].unsqueeze(2))

In [31]:
torch.cat((seq, output), 1)

tensor([[-0.3146, -0.3028, -0.2510,  ...,  0.1329,  0.0255,  0.0628],
        [-1.4087, -1.1531, -0.8216,  ..., -1.4326, -1.3465,  0.0716],
        [-0.3261, -0.3865, -0.4323,  ..., -0.2205, -0.2714,  0.0637],
        ...,
        [-1.2861, -1.4797, -1.6593,  ..., -0.9629, -1.0202,  0.0689],
        [ 0.8442,  0.7455,  0.6537,  ..., -0.0507, -0.0827,  0.0633],
        [ 0.7303,  0.2767,  0.0232,  ...,  0.6491,  0.5682,  0.0598]],
       grad_fn=<CatBackward>)

In [10]:
val_mse = AverageMeter()
loss = nn.MSELoss()
model.eval()
for i, (sequence, target) in enumerate(val_loader):
    batch_size = target.shape[0]
    look_back = sequence.shape[-1]
    sequence = sequence.unsqueeze(2).to(device)
    for step in range(target.shape[-1]):
        output = model(sequence[:,-look_back:,:])
        sequence = torch.cat((sequence, output.unsqueeze(2)), 1)
    val_mse.update(loss(sequence[:,-24:].cpu(), target.squeeze(1)), batch_size)
    
    break
    

/home/charles/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([16, 24])) that is different to the input size (torch.Size([16, 24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (24) must match the size of tensor b (16) at non-singleton dimension 1

In [52]:
crit = nn.MSELoss(reduction='sum')
SSE = torch.sum((sequence[:,-24:] - target.squeeze(1))**2)

In [53]:
SSE

tensor(504.5407, grad_fn=<SumBackward0>)

In [54]:
crit(sequence[:,-24:], target.squeeze(1))

tensor(504.5407, grad_fn=<MseLossBackward>)

In [9]:
dataloader = DataLoader(EnergyDemandDataset('val', 24), batch_size = 32, shuffle = True)
seq, target = next(iter(dataloader))

In [10]:
seq.shape

torch.Size([32, 24])

In [11]:
target.shape

torch.Size([32, 1, 24])

In [6]:
class EnergyDemandDataset(Dataset):
    """Energy Demand Dataset Object"""

    def __init__(self, split, look_back, transform = 'standard', directory = 'data'):
        """
        Args:
            split (string): Data split -> train, validation, or test'.
            look_back (int): Number of time-steps into the past for each sample
            transform (string): Data normalization type -> min-max or standardize
            directory (string): Directory where data is located
        """
        
        assert split in ['train', 'val', 'test'], "split needs to be either 'train', 'val', or 'test'"
        assert transform in ['min-max', 'standard'], "transform only supports 'min-max' or 'standard' (standard score)"
        assert isinstance(look_back, int) and look_back > 0, 'look_back needs to be integer greater than 0'
        
        self.look_back = look_back
        self.transform = transform
        self.split = split
        
        if self.split == 'train':
            self.data = pickle.load(open(os.path.join(directory,'{}.p'.format('train')), 'rb'))['MwH'].values
        elif self.split == 'val':
            past = pickle.load(open(os.path.join(directory,'{}.p'.format('train')), 'rb'))['MwH'].values
            present = pickle.load(open(os.path.join(directory,'{}.p'.format('validation')), 'rb'))['MwH'].values
            self.data = np.concatenate((past[-self.look_back:],present))
        else:
            past = np.concatenate((pickle.load(open(os.path.join(directory,'{}.p'.format('train')), 'rb'))['MwH'].values,
                                   pickle.load(open(os.path.join(directory,'{}.p'.format('validation')), 'rb'))['MwH'].values))
            present = pickle.load(open(os.path.join(directory,'{}.p'.format('test')), 'rb'))['MwH'].values
            self.data = np.concatenate((past[-self.look_back:],present))
        
        self.mean = 18251.291178385418
        self.std = 3475.1125327833797

        self.min = 11831.0
        self.max = 32076.0
        
        if self.transform == 'min-max':
            self.data = (self.data - self.min)/(self.max-self.min)
        elif self.transform == 'standard':
            self.data = (self.data - self.mean)/(self.std)
            
    def __len__(self):
        
        if self.split == 'train':
            return self.data.shape[0]-self.look_back
        else:
            return self.data.shape[0]-self.look_back-24

    def __getitem__(self, idx):
        
        if self.split == 'train':
            sequence = torch.Tensor(self.data[0+idx:self.look_back+idx])
            target = torch.Tensor([self.data[self.look_back+idx]])
        else:
            sequence = torch.Tensor(self.data[0+idx:self.look_back+idx])
            target = torch.Tensor([self.data[self.look_back+idx:self.look_back+idx+24]])
            
        return sequence, target

In [12]:
dataloader = DataLoader(EnergyDemandDataset('test', 24*7), batch_size = 32, shuffle = True)

In [13]:
seq, target = next(iter(dataloader))

In [14]:
seq.shape

torch.Size([32, 168])

In [15]:
target.shape

torch.Size([32, 1, 24])